In [87]:
names = open('pokemon_names.txt','r').read().splitlines()

In [88]:
# Cleaning the data

# Turning the words into lowercase
words = [w.lower() for w in names]

# Removing symbols, punctuations and numbers for the names
words = [word for word in words if not any(letter in word for letter in ['.','-',' ',"’",'♀','♂',"|"])]
words.remove('porygon2')
words = ['flabebe' if word == 'flabébé' else word for word in words]

In [89]:
chars = sorted(set(''.join(words)))

In [90]:
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [91]:
import torch

In [92]:
xs = []
ys = []
for w in words[:1]:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
xy = torch.tensor(ys)

In [93]:
g = torch.Generator().manual_seed(23021996)
W = torch.randn((27,27), generator = g, requires_grad=True)
num = xs.nelement()

In [94]:
import torch.nn.functional as F

In [95]:
for k in range(1):
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts/counts.sum(dim=1,keepdim=True)
    loss = -probs[torch.arange(num), ys].log().mean()

In [96]:
W.grad = None
loss.backward()

W.data += -50 * W.grad

In [97]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  
  out = []
  ix = 0
  while True:
    
    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------
    
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

dexbur.
bulbur.
bur.
vtxhkmdlbur.
bur.
